In [10]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import PreTrainedModel
from transformers.pipelines.pt_utils import KeyDataset

from tqdm.auto import tqdm

from datasets import Dataset, DatasetDict, load_dataset
from datasets import load_metric

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

import pandas as pd
import numpy as np
import logging
from glob import glob
from os import path

from IPython.display import HTML, display

import torch

In [11]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
device

device(type='cpu')

In [12]:
dataset_path = r"feedback-prize-2021/train.csv"
df = pd.read_csv(dataset_path, header=0, encoding= 'unicode_escape')
df = df.set_index("id")
df = df.rename(columns={"discourse_text": "text"})
df["discourse_type"] = pd.Categorical(df["discourse_type"])
df["label"] = df["discourse_type"].cat.codes
df

,discourse_id,discourse_start,discourse_end,text,discourse_type,discourse_type_num,predictionstring,label
id,,,,,,,,
423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,4
423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,5
423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,3
423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,3
423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,0
...,...,...,...,...,...,...,...,...
4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,3
4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,3
4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seekÂ multiple opinions instea...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,5


In [13]:
category_codes = dict(zip(range(df["discourse_type"].cat.categories.size),df["discourse_type"].cat.categories))
category_codes

{0: 'Claim',
 1: 'Concluding Statement',
 2: 'Counterclaim',
 3: 'Evidence',
 4: 'Lead',
 5: 'Position',
 6: 'Rebuttal'}

In [14]:
dataset = Dataset.from_pandas(df[["text", "label"]])

train_test_dataset = dataset.train_test_split(test_size=0.3)
test_validation_dataset = train_test_dataset["test"].train_test_split(test_size=0.333)

train_test_valid_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_validation_dataset['train'],
    'valid': test_validation_dataset['test']})

train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 101005
    })
    test: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 28873
    })
    valid: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 14415
    })
})

In [18]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 8
num_labels = len(category_codes)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [15]:
model_path = r"models_gitignored/distilbert-base-uncased-finetuned-sentence-classification/checkpoint-12626"
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path, id2label=category_codes)
loaded_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [16]:
articles_dataset = load_dataset("text", data_files={"valid":[r"datagen/nytimes/metaverse-politics-disinformation-society.txt"]})['valid']

# filter out empty strings
articles_dataset = articles_dataset.filter(lambda example: example['text'])

articles_dataset[1]

Using custom data configuration default-af442e383791019f


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to C:\Users\Prannaya\.cache\huggingface\datasets\text\default-af442e383791019f\0.0.0\08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'text': 'In the metaverse, a user might curate a digital avatar, like a character in a video game. Through the eyes of their avatar, they would experience a digital reality as active and engaging as the physical one. Some futurists believe that soon we might attend doctor’s appointments or class there.'}

In [19]:
pipe = pipeline("text-classification", model=loaded_model, tokenizer=tokenizer)
pipe

In [26]:
lst = []
for i, out in enumerate(tqdm(pipe(KeyDataset(articles_dataset, "text")))):
    print(articles_dataset[i]['text'], out)
    lst.append((articles_dataset[i]['text'], out))

  0%|          | 0/21 [00:00<?, ?it/s]

The metaverse is coming. It was once a science-fiction fantasy, most notably in Neal Stephenson’s novel “Snow Crash,” of an all-encompassing virtual universe that would exist alongside the physical one. But technological advances have brought this transformation of human society close enough to reality to demand that we consider its consequences. {'label': 'Lead', 'score': 0.9138175845146179}
In the metaverse, a user might curate a digital avatar, like a character in a video game. Through the eyes of their avatar, they would experience a digital reality as active and engaging as the physical one. Some futurists believe that soon we might attend doctor’s appointments or class there. {'label': 'Evidence', 'score': 0.4578322768211365}
But while the metaverse could revolutionize work and play, it is essential to remain wary of the dangers that will emerge if it subsumes daily life. {'label': 'Rebuttal', 'score': 0.8579508066177368}
Virtual environments will supercharge disinformation campa

In [30]:
df = pd.DataFrame([{"text":i, **j} for i, j in lst])

labels = ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement', 'Counterclaim', 'Rebuttal']
colors = ["#ACDDDE", "#CAF1DE", "#E1F8DC", "#FEF8DD", "#FFE7C7", "#F7D8BA", "#D6CDEA"]


s = "<h2>Legend</h2><br>"
for label, color in zip(labels, colors):
    s += "<font style=\"background-color: %s;\">%s</font><br>" % (color, label)
s += "<br/><br/><br/><h2>Text</h2><br>"

for txt,lbl in zip(df.text.tolist(),df.label.tolist()):
    
    s += "<font style=\"background-color: %s;\">%s </font>" % (colors[labels.index(lbl)], txt)
HTML(s+"<br/><br/><br/>")

In [33]:
glob("datagen/nytimes/*.txt")

['datagen/nytimes\\abortion-florida-15-week-ban.txt',
 'datagen/nytimes\\adhd-dating-relationships.txt',
 'datagen/nytimes\\afghanistan-boy-dies-well.txt',
 'datagen/nytimes\\afghanistan-immigration-visa-us.txt',
 'datagen/nytimes\\ahmaud-arbery-mcmichael-trial.txt',
 'datagen/nytimes\\ai-education-neural-networks.txt',
 'datagen/nytimes\\aids-pandemic-covid.txt',
 'datagen/nytimes\\allison-gollust-cnn-cuomo.txt',
 'datagen/nytimes\\american-girl-cafe-harry-hill-serena-kerrigan.txt',
 'datagen/nytimes\\andrew-prince-charles-charity.txt',
 'datagen/nytimes\\apple-face-computers.txt',
 'datagen/nytimes\\at-yosemite-a-waterfall-turns-into-a-firefall.txt',
 'datagen/nytimes\\australia-tourism-covid.txt',
 'datagen/nytimes\\babies-work-meeting.txt',
 'datagen/nytimes\\basketball-celtics-ime-udoka.txt',
 'datagen/nytimes\\basquiat-painting-orlando-mumford-museum.txt',
 'datagen/nytimes\\berlin-film-festival-2022.txt',
 'datagen/nytimes\\biden-economy-inflation-growth.txt',
 'datagen/nytimes\

In [35]:
dfs = []
for filename in glob("datagen/nytimes/*.txt"):
    articles_dataset = load_dataset("text", data_files={"valid":[filename]})['valid'].filter(lambda example: example['text'])
    lst = []
    for i, out in enumerate(tqdm(pipe(KeyDataset(articles_dataset, "text")))):
        print(articles_dataset[i]['text'], out)
        lst.append((articles_dataset[i]['text'], out))
    
    dfs.append(pd.DataFrame([{"text":i, **j} for i, j in lst]))

Using custom data configuration default-c7f59a738d9c36bc


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to C:\Users\Prannaya\.cache\huggingface\datasets\text\default-c7f59a738d9c36bc\0.0.0\08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

The network has spread videos emphasizing the Olympics as environmentally friendly and crooning about strengthening Chinese-Russian ties, punctuated by President Vladimir V. Putin’s attendance at the opening ceremony. {'label': 'Evidence', 'score': 0.7060900330543518}
China has defended its use of Twitter and Facebook, platforms that it bans at home. A foreign ministry spokeswoman, Hua Chunying, said last year that such sites were an “extra channel” to combat negative portrayals in the West. {'label': 'Evidence', 'score': 0.9439497590065002}
One American company, Vippi Media, based in New Jersey, signed a $300,000 contract with the consulate general of China in New York to help promote the Games, according to the company’s filing with the Justice Department under the Foreign Agents Registration Act. {'label': 'Evidence', 'score': 0.874495804309845}
Under the contract, first reported by the research group Open Secrets, the company has been promoting the Games by recruiting “social media

Using custom data configuration default-51dcca0881649558


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 96: invalid start byte

In [36]:
labels = ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement', 'Counterclaim', 'Rebuttal']
colors = ["#ACDDDE", "#CAF1DE", "#E1F8DC", "#FEF8DD", "#FFE7C7", "#F7D8BA", "#D6CDEA"]


s = "<h2>Legend</h2><br>"
for label, color in zip(labels, colors):
    s += "<font style=\"background-color: %s;\">%s</font><br>" % (color, label)
s += "<br/><br/><br/>"

for df in dfs:
    s += "<h2>Text</h2><br>"

    for txt,lbl in zip(df.text.tolist(),df.label.tolist()):

        s += "<font style=\"background-color: %s;\">%s </font>" % (colors[labels.index(lbl)], txt)
    s += "<br/><br/><hr><br/>"

HTML(s)